LSTM

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd drive/My\ Drive/FAKE_NEWS_DETECTION/DATASET

/content/drive/My Drive/FAKE_NEWS_DETECTION/DATASET


In [ ]:
!pip install --upgrade tensorflow-gpu==2.0

In [ ]:
pip install numpy==1.19.5

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import pandas as pd

In [7]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [8]:
from glob import glob
lis = glob('*.csv')
print(lis)

['KAGGLE_DATASET_Jruvika.csv', 'ISOT_FAKE_NEWS_DATASET.csv', 'UTK_FAKE_NEWS_DATASET.csv', 'G_McIntire_FAKE_NEWS_DATASET_.csv', 'LIAR_FAKE_NEWS_DATASET.csv', 'COVID-19_Tweets_FAKE_NEWS_DATASET.csv']


In [9]:
voc_size=5000
sent_length=20
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  

In [13]:
rows=[]
for i in lis :
  # read training data
  df_train = pd.read_csv(i)
  print("-------------------------")

  X_train = df_train['text']
  y_train = df_train['label']

  corpus =df_train['text']
  onehot_repr=[one_hot(str(words),voc_size)for words in corpus]
  
  embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
  X=np.array(embedded_docs)
  y=np.array(y_train)
  model.fit(X,y,epochs=5,validation_split=0.2,batch_size=64)
  j_compare=[]
  j_compare.append(i)
  for j in lis:

    df_test = pd.read_csv(j)

    X__test = df_test['text']
    y__test = df_test['label']
    onehot_repr1=[one_hot(str(words),voc_size)for words in X__test]
    embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=sent_length)
    X__=np.array(embedded_docs1)
    y__=np.array(y__test)
    y_pred=model.predict_classes(X__)
    # compute performance
    performances = np.asarray(precision_recall_fscore_support(y__,y_pred, average='macro'))
    j_compare.append(performances[2])
    # save results
    print(performances)
    print(i+j+"completed")
  rows.append(j_compare)
    

-------------------------
Train on 3310 samples, validate on 828 samples
Epoch 1/5
3310/3310 [==============================] - 3s 767us/sample - loss: 0.3037 - accuracy: 0.8695 - val_loss: 0.1823 - val_accuracy: 0.9275
Epoch 2/5
3310/3310 [==============================] - 3s 786us/sample - loss: 0.0906 - accuracy: 0.9692 - val_loss: 0.1475 - val_accuracy: 0.9432
Epoch 3/5
3310/3310 [==============================] - 3s 761us/sample - loss: 0.0362 - accuracy: 0.9903 - val_loss: 0.1405 - val_accuracy: 0.9553
Epoch 4/5
3310/3310 [==============================] - 2s 752us/sample - loss: 0.0147 - accuracy: 0.9970 - val_loss: 0.1434 - val_accuracy: 0.9577
Epoch 5/5
3310/3310 [==============================] - 3s 756us/sample - loss: 0.0049 - accuracy: 0.9997 - val_loss: 0.1568 - val_accuracy: 0.9626
[0.9922037990443502 0.9923455410737326 0.9922632965825898 None]
KAGGLE_DATASET_Jruvika.csvKAGGLE_DATASET_Jruvika.csvcompleted
[0.24019383016652776 0.3650502148001421 0.2791061604213356 None]
K

In [17]:
import csv
fields=['','KAGGLE_DATASET_Jruvika.csv', 'ISOT_FAKE_NEWS_DATASET.csv', 'UTK_FAKE_NEWS_DATASET.csv', 'G_McIntire_FAKE_NEWS_DATASET_.csv', 'LIAR_FAKE_NEWS_DATASET.csv', 'COVID-19_Tweets_FAKE_NEWS_DATASET.csv']
with open('../RESULT/LSTM_RESULT.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)